# Deep Q-Learning 

For this assignment we will implement the Deep Q-Learning algorithm with Experience Replay as described in breakthrough paper __"Playing Atari with Deep Reinforcement Learning"__. We will train an agent to play the famous game of __Breakout__.

In [1]:
import sys
import gym
import torch

import pylab
import random
import numpy as np
from collections import deque
from datetime import datetime
from copy import deepcopy
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from utils import *
from agent import *
from model import *
from config import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Understanding the environment

In the following cell, we initialise our game of __Breakout__ and you can see how the environment looks like. For further documentation of the of the environment refer to https://gym.openai.com/envs. 

In [2]:
env = gym.make('BreakoutDeterministic-v4')
# env.render()

/home/dl/downloads/venv3/lib/python3.7/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [3]:
number_lives = find_max_lifes(env)
state_size = env.observation_space.shape
action_size = 3
rewards, episodes = [], []

## Creating a DQN Agent

Here we create a DQN Agent. This agent is defined in the __agent.py__. The corresponding neural network is defined in the __model.py__. 

__Evaluation Reward__ : The average reward received in the past 100 episodes/games.

__Frame__ : Number of frames processed in total.

__Memory Size__ : The current size of the replay memory.

In [4]:
agent = Agent(action_size)
evaluation_reward = deque(maxlen=evaluation_reward_length)
frame = 0
memory_size = 0

### Main Training Loop

In [ ]:
for e in range(EPISODES):
    done = False
    score = 0

    history = np.zeros([5, 84, 84], dtype=np.uint8)
    step = 0
    d = False
    state = env.reset()
    life = number_lives

    get_init_state(history, state)

    while not done:
        step += 1
        frame += 1
        #if render_breakout:
        #    env.render()

        # Select and perform an action
        action = agent.get_action(np.float32(history[:4, :, :]) / 255.)

        
        next_state, reward, done, info = env.step(action + 1)

        frame_next_state = get_frame(next_state)
        history[4, :, :] = frame_next_state
        terminal_state = check_live(life, info['ale.lives'])

        life = info['ale.lives']
        r = np.clip(reward, -1, 1)

        # Store the transition in memory 
        agent.memory.push(deepcopy(frame_next_state), action, r, terminal_state)
        # Start training after random sample generation
        if(frame >= train_frame):
            agent.train_policy_net(frame)
            # Update the target network
            if(frame % Update_target_network_frequency)== 0:
                agent.update_target_net()
        score += reward
        history[:4, :, :] = history[1:, :, :]

        if frame % 50000 == 0:
            print('now time : ', datetime.now())
            rewards.append(np.mean(evaluation_reward))
            episodes.append(e)
            pylab.plot(episodes, rewards, 'b')
            pylab.savefig("./save_graph/breakout_dqn.png")

        if done:
            evaluation_reward.append(score)
            # every episode, plot the play time
            print("episode:", e, "  score:", score, "  memory length:",
                  len(agent.memory), "  epsilon:", agent.epsilon, "   steps:", step,
                  "    evaluation reward:", np.mean(evaluation_reward))

            # if the mean of scores of last 10 episode is bigger than 400
            # stop training
            if np.mean(evaluation_reward) > 10:
                torch.save(agent.policy_net, "./save_model/breakout_dqn")
                sys.exit()

/home/dl/downloads/venv3/lib/python3.7/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


episode: 0   score: 2.0   memory length: 196   epsilon: 1.0    steps: 196     evaluation reward: 2.0
episode: 1   score: 3.0   memory length: 457   epsilon: 1.0    steps: 261     evaluation reward: 2.5
episode: 2   score: 1.0   memory length: 630   epsilon: 1.0    steps: 173     evaluation reward: 2.0
episode: 3   score: 1.0   memory length: 784   epsilon: 1.0    steps: 154     evaluation reward: 1.75
episode: 4   score: 2.0   memory length: 988   epsilon: 1.0    steps: 204     evaluation reward: 1.8
episode: 5   score: 0.0   memory length: 1123   epsilon: 1.0    steps: 135     evaluation reward: 1.5
episode: 6   score: 2.0   memory length: 1352   epsilon: 1.0    steps: 229     evaluation reward: 1.5714285714285714
episode: 7   score: 2.0   memory length: 1548   epsilon: 1.0    steps: 196     evaluation reward: 1.625
episode: 8   score: 0.0   memory length: 1673   epsilon: 1.0    steps: 125     evaluation reward: 1.4444444444444444
episode: 9   score: 0.0   memory length: 1806   epsilo

episode: 72   score: 1.0   memory length: 13279   epsilon: 1.0    steps: 161     evaluation reward: 1.273972602739726
episode: 73   score: 2.0   memory length: 13481   epsilon: 1.0    steps: 202     evaluation reward: 1.2837837837837838
episode: 74   score: 2.0   memory length: 13679   epsilon: 1.0    steps: 198     evaluation reward: 1.2933333333333332
episode: 75   score: 1.0   memory length: 13834   epsilon: 1.0    steps: 155     evaluation reward: 1.2894736842105263
episode: 76   score: 0.0   memory length: 13970   epsilon: 1.0    steps: 136     evaluation reward: 1.2727272727272727
episode: 77   score: 1.0   memory length: 14147   epsilon: 1.0    steps: 177     evaluation reward: 1.2692307692307692
episode: 78   score: 2.0   memory length: 14349   epsilon: 1.0    steps: 202     evaluation reward: 1.2784810126582278
episode: 79   score: 0.0   memory length: 14484   epsilon: 1.0    steps: 135     evaluation reward: 1.2625
episode: 80   score: 2.0   memory length: 14702   epsilon: 1.

episode: 147   score: 1.0   memory length: 27328   epsilon: 1.0    steps: 179     evaluation reward: 1.34
episode: 148   score: 1.0   memory length: 27502   epsilon: 1.0    steps: 174     evaluation reward: 1.33
episode: 149   score: 1.0   memory length: 27661   epsilon: 1.0    steps: 159     evaluation reward: 1.33
episode: 150   score: 2.0   memory length: 27847   epsilon: 1.0    steps: 186     evaluation reward: 1.35
episode: 151   score: 0.0   memory length: 27988   epsilon: 1.0    steps: 141     evaluation reward: 1.33
episode: 152   score: 0.0   memory length: 28113   epsilon: 1.0    steps: 125     evaluation reward: 1.31
episode: 153   score: 2.0   memory length: 28318   epsilon: 1.0    steps: 205     evaluation reward: 1.31
episode: 154   score: 2.0   memory length: 28508   epsilon: 1.0    steps: 190     evaluation reward: 1.33
episode: 155   score: 2.0   memory length: 28713   epsilon: 1.0    steps: 205     evaluation reward: 1.34
episode: 156   score: 0.0   memory length: 288

episode: 225   score: 2.0   memory length: 41221   epsilon: 1.0    steps: 180     evaluation reward: 1.21
episode: 226   score: 0.0   memory length: 41358   epsilon: 1.0    steps: 137     evaluation reward: 1.21
episode: 227   score: 3.0   memory length: 41584   epsilon: 1.0    steps: 226     evaluation reward: 1.23
episode: 228   score: 2.0   memory length: 41789   epsilon: 1.0    steps: 205     evaluation reward: 1.24
episode: 229   score: 0.0   memory length: 41923   epsilon: 1.0    steps: 134     evaluation reward: 1.21
episode: 230   score: 0.0   memory length: 42065   epsilon: 1.0    steps: 142     evaluation reward: 1.2
episode: 231   score: 1.0   memory length: 42240   epsilon: 1.0    steps: 175     evaluation reward: 1.21
episode: 232   score: 3.0   memory length: 42461   epsilon: 1.0    steps: 221     evaluation reward: 1.21
episode: 233   score: 2.0   memory length: 42660   epsilon: 1.0    steps: 199     evaluation reward: 1.23
episode: 234   score: 0.0   memory length: 4280

...

episode: 1817   score: 9.0   memory length: 389872   epsilon: 0.7130257300062299    steps: 334     evaluation reward: 4.69
episode: 1818   score: 6.0   memory length: 390252   epsilon: 0.7126495300062381    steps: 380     evaluation reward: 4.67
episode: 1819   score: 10.0   memory length: 390756   epsilon: 0.7121505700062489    steps: 504     evaluation reward: 4.7
episode: 1820   score: 5.0   memory length: 391043   epsilon: 0.7118664400062551    steps: 287     evaluation reward: 4.72
episode: 1821   score: 4.0   memory length: 391334   epsilon: 0.7115783500062614    steps: 291     evaluation reward: 4.73
episode: 1822   score: 8.0   memory length: 391785   epsilon: 0.711131860006271    steps: 451     evaluation reward: 4.71
episode: 1823   score: 3.0   memory length: 392029   epsilon: 0.7108903000062763    steps: 244     evaluation reward: 4.68
episode: 1824   score: 8.0   memory length: 392348   epsilon: 0.7105744900062831    steps: 319     evaluation reward: 4.74
episode: 1825   s

episode: 1884   score: 6.0   memory length: 412573   epsilon: 0.6905517400067178    steps: 359     evaluation reward: 5.55
episode: 1885   score: 7.0   memory length: 412934   epsilon: 0.6901943500067256    steps: 361     evaluation reward: 5.61
episode: 1886   score: 6.0   memory length: 413277   epsilon: 0.689854780006733    steps: 343     evaluation reward: 5.65
episode: 1887   score: 6.0   memory length: 413613   epsilon: 0.6895221400067402    steps: 336     evaluation reward: 5.65
episode: 1888   score: 10.0   memory length: 414038   epsilon: 0.6891013900067493    steps: 425     evaluation reward: 5.67
episode: 1889   score: 9.0   memory length: 414369   epsilon: 0.6887737000067564    steps: 331     evaluation reward: 5.73
episode: 1890   score: 4.0   memory length: 414652   epsilon: 0.6884935300067625    steps: 283     evaluation reward: 5.74
episode: 1891   score: 2.0   memory length: 414842   epsilon: 0.6883054300067666    steps: 190     evaluation reward: 5.7
episode: 1892   s

episode: 1951   score: 6.0   memory length: 434109   epsilon: 0.6692311000071807    steps: 400     evaluation reward: 5.5
episode: 1952   score: 2.0   memory length: 434301   epsilon: 0.6690410200071848    steps: 192     evaluation reward: 5.49
episode: 1953   score: 2.0   memory length: 434508   epsilon: 0.6688360900071892    steps: 207     evaluation reward: 5.49
episode: 1954   score: 9.0   memory length: 434977   epsilon: 0.6683717800071993    steps: 469     evaluation reward: 5.54
episode: 1955   score: 14.0   memory length: 435612   epsilon: 0.667743130007213    steps: 635     evaluation reward: 5.62
episode: 1956   score: 17.0   memory length: 436110   epsilon: 0.6672501100072237    steps: 498     evaluation reward: 5.74
episode: 1957   score: 4.0   memory length: 436370   epsilon: 0.6669927100072293    steps: 260     evaluation reward: 5.72
episode: 1958   score: 4.0   memory length: 436640   epsilon: 0.6667254100072351    steps: 270     evaluation reward: 5.72
episode: 1959   

episode: 2018   score: 4.0   memory length: 457915   epsilon: 0.6456631600076923    steps: 272     evaluation reward: 6.13
episode: 2019   score: 3.0   memory length: 458136   epsilon: 0.645444370007697    steps: 221     evaluation reward: 6.11
episode: 2020   score: 9.0   memory length: 458582   epsilon: 0.6450028300077066    steps: 446     evaluation reward: 6.09
episode: 2021   score: 6.0   memory length: 458932   epsilon: 0.6446563300077142    steps: 350     evaluation reward: 6.05
episode: 2022   score: 11.0   memory length: 459347   epsilon: 0.6442454800077231    steps: 415     evaluation reward: 6.13
episode: 2023   score: 3.0   memory length: 459573   epsilon: 0.6440217400077279    steps: 226     evaluation reward: 6.1
episode: 2024   score: 11.0   memory length: 460005   epsilon: 0.6435940600077372    steps: 432     evaluation reward: 6.2
episode: 2025   score: 13.0   memory length: 460612   epsilon: 0.6429931300077503    steps: 607     evaluation reward: 6.3
episode: 2026   s

episode: 2085   score: 3.0   memory length: 482525   epsilon: 0.6212992600082212    steps: 265     evaluation reward: 7.02
episode: 2086   score: 9.0   memory length: 483002   epsilon: 0.6208270300082315    steps: 477     evaluation reward: 7.07
episode: 2087   score: 12.0   memory length: 483463   epsilon: 0.6203706400082414    steps: 461     evaluation reward: 7.12
episode: 2088   score: 10.0   memory length: 483915   epsilon: 0.6199231600082511    steps: 452     evaluation reward: 7.13
episode: 2089   score: 3.0   memory length: 484164   epsilon: 0.6196766500082564    steps: 249     evaluation reward: 7.11
episode: 2090   score: 3.0   memory length: 484429   epsilon: 0.6194143000082621    steps: 265     evaluation reward: 7.08
episode: 2091   score: 5.0   memory length: 484754   epsilon: 0.6190925500082691    steps: 325     evaluation reward: 7.04
episode: 2092   score: 8.0   memory length: 485213   epsilon: 0.618638140008279    steps: 459     evaluation reward: 7.11
episode: 2093  

In [ ]:
torch.save(agent.policy_net, "./save_model/breakout_dqn")